<a href="https://colab.research.google.com/github/Anchayadav/AutoTS/blob/main/End_to_End_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=175466b090a84510e6a1814a949caf22161d246cc1d4862bc152dc96986aba49
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pandas as pd
data=pd.read_csv('fashion_products.csv')

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [ ]:
data1=spark.read.csv("fashion_products.csv")

In [ ]:
data1=spark.read.csv('fashion_products.csv',header=True,inferSchema=True).show()

+-------+----------+------------+------+---------------+-----+------------------+------+----+
|User ID|Product ID|Product Name| Brand|       Category|Price|            Rating| Color|Size|
+-------+----------+------------+------+---------------+-----+------------------+------+----+
|     19|         1|       Dress|Adidas|  Men's Fashion|   40|1.0431592108361825| Black|  XL|
|     97|         2|       Shoes|   H&M|Women's Fashion|   82| 4.026416271141911| Black|   L|
|     25|         3|       Dress|Adidas|Women's Fashion|   44| 3.337937559377053|Yellow|  XL|
|     57|         4|       Shoes|  Zara|  Men's Fashion|   23|1.0495229563128543| White|   S|
|     79|         5|     T-shirt|Adidas|  Men's Fashion|   79| 4.302773408398684| Black|   M|
|     98|         6|       Dress|Adidas|  Men's Fashion|   47|1.3795657395330458|Yellow|   L|
|     16|         7|       Jeans| Gucci|  Men's Fashion|   37|1.3567503746842564| White|  XL|
|     63|         8|     Sweater|  Zara|  Kids' Fashion|   6

In [ ]:
data1.head(5)

AttributeError: ignored

In [ ]:
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
ssl._create_default_https_context=ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
intents = [
    {
        "tag":"greeting",
        "patterns":["hi","hellow","hey","how","how are you","what is up"],
        "responses":["hi there","hellow","hey","I'm fine,thankl you","Nothing much"]
    },
    {
        "tag":"goodby",
        "patterns":["bay","see you latter","goodby","take care"],
        "responses":["goodby","see you later","take care"]
    },
    {
        "tag":"weather",
        "patterns":["what's the weather like","how's the weather today"],
        "responses":["I'm sorry I cannot provide real-time weather information,","you can check the weather on a weather app or website."]
    }
]

In [ ]:
vectorizer = TfidfVectorizer()
clf=LogisticRegression(random_state=0,max_iter=10000)
tags = []
patterns =[]
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)
x=vectorizer.fit_transform(patterns)
y=tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [ ]:
def chatbot(input_text):
  input_text=vectorizer.transform(input_text)
  tag = clf.predict(input_text)[0]
  for intent in intents:
    if intent['tag'] == tag:
      response =random.choice(intent['responses'])
      return response



In [ ]:
counter =0
def main():
  global counter
  st.title("chatbot")
  st.write("welcome to the chatbot. please  type a message and press enter to start the conversation.")

  counter +=1
  user_input = st.text_input("you" ,key=f"user_input_{counter}")

  if user_input:
    response = chatbot(user_input)
    st.text_area("chatbot:",value=response,height=100,max_chars=None,key=f"chatbot_response_{counter}")

    if response.lower() in ['goodby','bye']:
      st.write("thank you for  cahtting with me. have a graet day."),
      st.stop()


if __name__=='__main__':
    main()


NameError: ignored